In [1]:
import cv2
import mediapipe as mp
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [2]:
def createBox(img,points, scale =5, masked = False , cropped = True):
    if masked:
        mask = np.zeros_like(img)
        mask=cv2.fillPoly(mask,[points],(255,255,255))
        img = cv2.bitwise_and(img,mask)
        #cv2.imshow('Mask',img)
    if cropped:
        bbox = cv2.boundingRect(points)
        x,y,w,h = bbox
        imgCrop = img[y:y+h,x:x+w]
        imgCrop = cv2.resize(imgCrop,(0,0),None,scale,scale)
        return imgCrop
    else:
        return mask
    

In [3]:
lips= [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291,375, 321, 405, 314, 17, 84, 181, 91, 146, 61,78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308,324, 318, 402, 317, 14, 87, 178, 88, 95, 78]
#lips = [61, 185, 40, 39, 37, 267, 269, 270, 409, 291,375, 321, 405, 314, 84,181, 91, 146,0]
#right_eye =[464,463,444,362,368,286,]
#eft_eye = [130,35,247,246,161,30,160,29,27,159,28,158,159,28,158,56,157,190,173,243,133]

In [4]:
# For webcam input:
mypoints=[]
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if results.multi_face_landmarks:
        mypoints=[]
        for face_landmarks in results.multi_face_landmarks:
            for lm_id, lm in enumerate(face_landmarks.landmark):
                h,w,c = image.shape
                x,y = int(lm.x*w),int(lm.y*h)
                points = [x,y]

                if lm_id in lips:
    #                 cv2.putText(
    #                 image,
    #                 str(lm_id),
    #                 (x,y),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 
    #                 0.3,
    #                 (0,0,255),
    #                 1)
                    mypoints.append(points)
        mypoints = np.array(mypoints)
      #print(mypoints)
        imglips = createBox(image,mypoints,masked=True,cropped=False)
        
        imgcolorlips = np.zeros_like(imglips)
        imgcolorlips[:] = 153,8,157
        imgcolorlips = cv2.bitwise_and(imglips,imgcolorlips)
        imgcolorlips = cv2.GaussianBlur(imgcolorlips,(7,7),10)
        imgcolorlips = cv2.addWeighted(image,1,imgcolorlips,1,0)
        cv2.imshow('colored', imgcolorlips)
        cv2.imshow('lips',imglips)
        
    

        
#         mp_drawing.draw_landmarks(
#             image=image,
#             landmark_list=face_landmarks,
#             connections=mp_face_mesh.FACEMESH_TESSELATION,
#             landmark_drawing_spec=None,
#             connection_drawing_spec=mp_drawing_styles
#             .get_default_face_mesh_tesselation_style())
        
        
#         mp_drawing.draw_landmarks(
#             image=image,
#             landmark_list=face_landmarks,
#             connections=mp_face_mesh.FACEMESH_CONTOURS,
#             landmark_drawing_spec=None,
#             connection_drawing_spec=mp_drawing_styles
#             .get_default_face_mesh_contours_style())
#         mp_drawing.draw_landmarks(
#             image=image,
#             landmark_list=face_landmarks,
#             connections=mp_face_mesh.FACEMESH_IRISES,
#             landmark_drawing_spec=None,
#             connection_drawing_spec=mp_drawing_styles
#             .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()